# Kassa Jeugdhuis Den Couter

In [1]:
filename = '../input/kassa210907/EJFILE.SDA'


import pandas as pd
df = pd.read_csv(filename, delimiter = "\r", header=None, names=['lines'])
df.head(10)

,lines
0,#000040 15/07/2021 22:51
1,01 CLERK01 000000
2,* S D*
3,S V F O N L E J
4,SD FINAL
5,#000041 15/07/2021 22:52
6,01 CLERK01 000000
7,1x 2.00 *2.00
8,Carlsberg
9,Carlsberg V-2.00


## print datumlijnen

In [2]:
count = 0
limit = 5

for index, row in df.iterrows():
    
    if (row['lines'][0] == '#') & (count<limit):
        count = count + 1
        print(row['lines'])


#000040   15/07/2021   22:51  
#000041   15/07/2021   22:52  
#000042   16/07/2021   14:32  
#000043   16/07/2021   14:36  
#000044   16/07/2021   14:36  


## print cashlijnen

In [3]:
from decimal import Decimal

count = 0
limit = 5

for index, row in df.iterrows():
    
    if ('CASH' in row['lines']) & (count<limit):
        count = count + 1
        amount = row['lines'].strip('CASH *').replace(' ','')
        amount = Decimal(amount)
        print(amount)

0.00
4.00
4.00
3.00
6.50


## print per blok

In [4]:
def start_block (row):
    if row['lines'][0] == '#' :
        return True
    return False

In [5]:
df['start_block'] = df.apply (lambda row: start_block(row), axis=1)
df['block_nr'] = df['start_block'].cumsum()
df.head(50)

,lines,start_block,block_nr
0,#000040 15/07/2021 22:51,True,1
1,01 CLERK01 000000,False,1
2,* S D*,False,1
3,S V F O N L E J,False,1
4,SD FINAL,False,1
5,#000041 15/07/2021 22:52,True,2
6,01 CLERK01 000000,False,2
7,1x 2.00 *2.00,False,2
8,Carlsberg,False,2
9,Carlsberg V-2.00,False,2


In [6]:

import datetime
from decimal import Decimal

MAX = df['block_nr'].max()

result_df = pd.DataFrame(columns=('id', 'datetime', 'amount'))
drinks_df = pd.DataFrame(columns=('id', 'drink', 'quantity','price'))

for i in range(1,MAX+1):
    block_df = df[df['block_nr'] == i]
    
    # check if it is a CASH block
    if 'CASH' in block_df.iloc[-1]['lines']:
        
        # get ID
        transaction_id = block_df.iloc[0]['lines']
        transaction_id = int(transaction_id[1:7])
        #print(transaction_id)
        
        # get date & time    
        dateAndtime = block_df.iloc[0]['lines'].split()
        date = dateAndtime[1]
        time = dateAndtime[2]
        date_time_str = date+' '+time
        date_time_obj = datetime.datetime.strptime(date_time_str, '%d/%m/%Y %H:%M')
        #print(date_time_obj)
        #print(date_time_obj.date())
        
        # get cash amount
        amount = block_df.iloc[-1]['lines'].strip('CASH *').replace(' ','')
        amount = Decimal(amount)
        #print(amount)
        
        # add to result_df
        result_df = result_df.append({'id':transaction_id, 'datetime':date_time_obj, 'amount': amount}, ignore_index=True)
        
        # --------------------------------
        
        block_iterator = block_df.iterrows()
        for index, row in block_iterator:
            if ('*' in row['lines']) & ('CASH' not in row['lines']):
                index2, row2 = next(block_iterator)
                
                # get quantity
                quantity = int(row['lines'].split('x')[0])
                #print(quantity)
                
                # get price
                price = Decimal(row['lines'].split('*')[-1])
                #print(price)
                
                # get drink name
                drink = row2['lines']
                #print(drink)

                drinks_df = drinks_df.append({'id':transaction_id, 'drink':drink, 'quantity': quantity, 'price': price}, ignore_index=True)

In [7]:
drinks_df.head()

,id,drink,quantity,price
0,41,Carlsberg,1,2.00
1,44,Export,1,2.00
2,44,Export,1,2.00
3,45,Export,1,2.00
4,45,Export,1,2.00


In [13]:
result_df

,id,datetime,amount
0,41,2021-07-15 22:52:00,0.00
1,44,2021-07-16 14:36:00,4.00
2,45,2021-07-16 16:42:00,4.00
3,46,2021-07-16 16:50:00,3.00
4,47,2021-07-16 17:23:00,6.50
...,...,...,...
886,1386,2021-09-05 03:58:00,0.50
887,1387,2021-09-05 04:04:00,0.50
888,1388,2021-09-05 04:08:00,3.50
889,1389,2021-09-05 04:09:00,1.50


# Omzet per dag

In [9]:

per_day_df = result_df.groupby(pd.to_datetime(result_df.datetime).dt.date).agg({'amount': 'sum'}).reset_index() 

per_day_df

,datetime,amount
0,2021-07-15,0.00
1,2021-07-16,50.00
2,2021-07-17,21.50
3,2021-07-18,6.00
4,2021-07-23,104.50
5,2021-07-24,178.50
6,2021-07-25,52.00
7,2021-07-30,583.00
8,2021-07-31,130.00
9,2021-08-07,15.00


In [14]:
drink_names = drinks_df['drink'].unique()
drink_quantities = pd.DataFrame(data=drink_names, columns=['drink'])
drink_quantities['quantity'] = 0

drink_quantities.head(5)

,drink,quantity
0,Carlsberg,0
1,Export,0
2,Primus,0
3,Cola,0
4,ros� wijn,0


In [16]:
quantities_per_drink = drinks_df.groupby(drinks_df['drink']).agg({'quantity': 'sum'}).reset_index() 
quantities_per_drink.head(5)

,drink,quantity
0,nutjes,18
1,BVDM,81
2,BVDM 3 + 1,4
3,BVHS,29
4,Bellini,48


In [17]:
quantities_per_drink['percent'] = ((quantities_per_drink['quantity']/uwu['quantity'].sum()).round(3)*100)
quantities_per_drink.sort_values('quantity', ascending=False)

,drink,quantity,percent
18,Primus,939,38.1
6,Carlsberg,223,9.1
8,Cola,140,5.7
19,Rodenbach,121,4.9
5,Bernadette,101,4.1
15,Kasteel Rouge,99,4.0
10,Duvel,81,3.3
1,BVDM,81,3.3
13,Fuzetea,64,2.6
22,Strongbow,63,2.6


# 